In [20]:
import mlrun
import os

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [21]:
TARGET_CODE_PATH = '/User/repos/functions/datagen/classification'
N_SAMPLES = 100_000
M_FEATURES = 28
NEG_WEIGHT = 0.5
TARGET_DATA_PATH = '/User/mlrun/models'
KEY = 'simdata'

In [22]:
testfn = mlrun.code_to_function(
    filename=os.path.join(TARGET_CODE_PATH, 'binary.py'), 
    kind='job')
testfn.build_config(base_image='yjbds/mlrun-ds:latest')
testfn.export(os.path.join(TARGET_CODE_PATH, 'binary.yaml'))

[mlrun] 2020-01-26 14:32:06,367 function spec saved to path: /User/repos/functions/datagen/classification/binary.yaml


In [23]:
binarydatagen = mlrun.import_function(
    os.path.join(TARGET_CODE_PATH, 'binary.yaml')
).apply(mlrun.mount_v3io())

In [24]:
binarydatagen.deploy(skip_deployed=True, with_mlrun=False)

[mlrun] 2020-01-26 14:32:09,075 starting remote build, image: .mlrun/func-default-binary-latest


True

In [25]:
task = mlrun.NewTask()
task.with_params(
    n_samples=N_SAMPLES,
    m_features=M_FEATURES,
    weight=NEG_WEIGHT,
    target_path=TARGET_DATA_PATH,
    key=KEY)

binarydatagen.run(task, handler='create_binary_classification')

[mlrun] 2020-01-26 14:32:15,900 starting run create_binary_classification uid=9dd358cd04554c9aa138275c0ec080aa  -> http://mlrun-api:8080
[mlrun] 2020-01-26 14:32:15,989 Job is running in the background, pod: create-binary-classification-crlf9
[mlrun] 2020-01-26 14:32:26,865 log artifact simdata at /User/mlrun/models/simdata-1e05X28.parquet, size: None, db: Y

[mlrun] 2020-01-26 14:32:26,877 run executed, status=completed
/opt/conda/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...c080aa,0,Jan 26 14:32:26,completed,binary,host=create-binary-classification-crlf9kind=jobowner=admin,,key=simdatam_features=28n_samples=100000target_path=/User/mlrun/modelsweight=0.5,,simdata


to track results use .show() or .logs() or in CLI: 
!mlrun get run 9dd358cd04554c9aa138275c0ec080aa  , !mlrun logs 9dd358cd04554c9aa138275c0ec080aa 
[mlrun] 2020-01-26 14:32:35,166 run executed, status=completed


# tests

In [26]:
import pandas as pd
fn =  f"simdata-{N_SAMPLES:0.0e}X{M_FEATURES}.parquet".replace("+", "")
df = pd.read_parquet(os.path.join(TARGET_DATA_PATH, fn), engine='pyarrow')
assert df.shape == (N_SAMPLES, M_FEATURES + 1)